# Risk Ratings  
This file processes the `Final Dataset v12` excel sheet to assign risk factors and risk ratings to purchases. It outputs a new excel sheet named `Risk_Result` which contains the risk ratings per purchase.

Running this file requires the follow modules to be installed: 
1. pandas

In [1]:
### Uncomment to install each dependency ###
#% pip install pandas

In [2]:
import pandas as pd

<b>po_checker</b>, <b>bos_checker</b>, and <b>inv_checker</b> check if the purchase order, bill of sale, and <br>
invoice documentation exists, respectively <br><br>

In [3]:
def po_checker(po):
    return po.isna().any(axis=1)

def bos_checker(bos):
    return bos.isna().any(axis=1)

def inv_checker(inv):
    return inv.isna().any(axis=1)

In [4]:
data = pd.read_excel('Processed_Data.xlsx')

# preliminary processing
bos_columns = data[['BOS Quantity', 'BOS Cost Per Unit', 'BOS Date']]
inv_columns = data[['Invoice Number', 'Invoice Group', 'Invoice Amount Paid', 
                          'Invoice Description', 'Invoice Date','Original Invoice Amount', 
                          'Invoice Quantity', 'Invoice Cost Per Unit']]
po_columns = data[['PO Number', 'PO Date', 'PO Quantity', 'PO Cost Per Unit']]

bos_exist = bos_checker(bos_columns)
inv_exist = inv_checker(inv_columns)
po_exist = po_checker(po_columns)

**a_checker** checks if the accounting code is missing or incorrect <br><br>

In [5]:
def a_checker(a):
    return pd.isna(a)

<b>b_checker</b> checks if the events 1 through 15 above happen out of order <br>
(ex: invoice date is prior to purchase order date) <br><br>

In [6]:
def b_checker(d):
    po, bos, inv = d['PO Date'], d['BOS Date'], d['Invoice Date']
    return (po > bos) | (bos > inv)

<b>c_checker</b> checks if the bill of sale or delivery documents are missing <br>
or do not agree with amounts on invoice<br><br>

In [7]:
def c_checker(b, d):
    bosQ, bosC = d['BOS Quantity'], d['BOS Cost Per Unit']
    invQ, invC = d['Invoice Quantity'], d['Invoice Cost Per Unit']
    return b | (bosQ != invQ) | (bosC != invC)

<b>d_checker</b> checks if the bill of sale or delivery documents are missing <br>
or do not agree with purchase order<br><br>

In [8]:
def d_checker(b, d):
    bosQ, bosC = d['BOS Quantity'], d['BOS Cost Per Unit']
    purQ, purC = d['PO Quantity'], d['PO Cost Per Unit']
    return b | (bosQ != purQ) | (bosC != purC)

<b>e_checker</b> checks if purchase order quantity does not agree with invoice quantity<br><br>

In [9]:
def e_checker(pur, inv):
    return pur != inv

<b>g_checker</b> checks if control documents or elements of the audit trail <br>
are missing: purchase order, bill of sale, invoice, accounting code<br><br>

In [10]:
def g_checker(p, b, i, acc):
    return p | b | i | acc.apply(pd.isna)

<b>h_checker</b> assigns appropriate risk ratings to transactions based on department characteristics.<br><br>

In [11]:
def h_checker(df):
    result = df['Department Name'].map(department_mapping) * 3.3
    return result.fillna(33)

<b>i_checker</b> checks for duplicate payments – <br>
same payments to same vendor for same good/service or <br>
same payment for same service/good to two or more different vendors<br><br>

In [12]:
def i_checker(df):
    # Initialize the duplicate flags column
    debug_copy = df.copy(deep=True)
    debug_copy['duplicate_flag'] = False
    
    # Group by clusters and check for duplicates within each cluster
    for cluster_id, cluster_df in debug_copy.groupby('cluster'):
        if cluster_id == -1:
            continue

        # Mark duplicates within the cluster based on 'Invoice Amount Paid'
        duplicates = cluster_df.duplicated('Invoice Amount Paid', keep=False)

        # Filter to keep only rows with different 'Check Number' and same 'Invoice Amount Paid'
        duplicate_pairs = cluster_df[duplicates]
        for invoice_amount, group in duplicate_pairs.groupby('Invoice Amount Paid'):
            if len(group['Check Number'].unique()) > 1:  # Ensure different check numbers
                debug_copy.loc[group.index, 'duplicate_flag'] = True
                
#     debug_copy.to_excel('debug.xlsx')

    return debug_copy['duplicate_flag']

<b> j_checker </b> checks for payments 2% under the threshold for competitive bidding <br>
  
The thresholds are as follows:  
Informal Bid Threshold: Above \\$10,000  
Formal Bid Threshold: Above \\$100,000  

In [13]:
def j_checker(df, column_name, informal_bid_thresh, formal_bid_thresh):
    def j_helper(value):
        if ((0.98 * informal_bid_thresh) <= value <= informal_bid_thresh or 
         
            (0.98 * formal_bid_thresh) <= value <= formal_bid_thresh):
            return True
        
        else:
            return False
    
    return df[column_name].apply(j_helper)

<b>k_checker</b> checks for payments 2% under the threshold of higher or Chief approval<br><br>
The thresholds are as follows:  
Supervisor: Less than or equal to \\$999  
Manager: \\$1000 - \\$4999  
Director: \\$5000 - \\$14999  
Senior Director: \\$15000 - \\$34999  
Executive Director/VP: \\$35000 - \\$49999  
Chief Officer: \\$50000 - \\$99999  
CFO: Greater than or equal to \\$100000  

In [14]:
def k_checker(df, column_name, threshold1, threshold2, threshold3, threshold4, threshold5, threshold6):
    def k_helper (value):
        if ((0.98 * threshold1) <= value <= threshold1 or
            
         (0.98 * threshold2) <= value <= threshold2 or
           
         (0.98 * threshold3) <= value <= threshold3 or
            
         (0.98 * threshold4) <= value <= threshold4 or
            
         (0.98 * threshold5) <= value <= threshold5 or
            
         (0.98 * threshold6) <= value <= threshold6):
            return True
        
        else:
            return False
        
    return df[column_name].apply(k_helper)

<b>l_checker</b> checks for payments to a vendor for the same service/good that accumulate<br>
to be in excess of blanket purchase order or contract<br><br>

<b>m_checker</b> checks for split purchase order - multiple POs to same vendor for same items<br><br>

**n_checker** split payments - payments to same vendor for same item and same date  

In [17]:
def n_checker(df):
    debug_copy = df.copy(deep=True)
    grouped = df.groupby(['Check Number','Supplier Number','Invoice Date'])
    debug_copy['Total Invoice amount'] = grouped['Invoice Amount Paid'].transform('sum')
    debug_copy['match'] = debug_copy['Payment Amount'] == debug_copy['Total Invoice amount']
    return ~debug_copy['match']

**o_checker** checks if vendor paid not on approved vendor list  

In [18]:
# compiled_active_app_vendors = pd.read_excel("compiled_active_app_vendors.xlsx", header = 1)
# def o_checker(df):
#     return df['Supplier Number'].isin(compiled_active_app_vendors['Supplier Code'])

In [19]:
department_mapping = {
    "Department 1": 1, "Department 2": 2, "Department 3": 3, "Department 4": 4,"Department 5": 5, 
    "Department 6": 2, "Department 2": 8, "Department 2": 2, "Department 2": 2, "Department 2": 8
}

descriptions = {
    'a': 'accounting code is missing or incorrect',
    'b': 'the events 1 through 15 above happen out of order (ex: invoice date is prior to purchase order date)',
    'c': 'bill of sale or delivery documents are missing or do not agree with amounts on invoice',
    'd': 'bill of sale or delivery documents are missing or do not agree with purchase order',
    'e': 'purchase order quantity does not agree with invoice quantity',
    'g': 'control documents or elements of the audit trail are missing: purchase order, bill of sale, invoice, accounting code',
    'h': 'the audit trail of transactions within a Department (Marketing, IT, U.M.) are inconsistent',
    'i': 'duplicate payments – same payments to same vendor for same good/service or same payment for same service/good to two or more different vendors',
    'j': 'payments 2% under the threshold for competitive bidding',
    'k': 'payments 2% under the threshold of higher or Chief approval',
    'l': 'payments to a vendor for the same service/good that accumulate to be in excess of blanket purchase order or contract',
    'm': 'split purchase order - multiple POs to same vendor for same items',
    'n': 'split payments - payments to same vendor for same item and same date',
    'o': 'vendor paid not on approved vendor list',
}

**map_descriptions**: This function maps descriptions to their corresponding letters

In [20]:
def map_descriptions(letters):
    letters = list(letters)
    mapped_descriptions = ['-' + descriptions[l] for l in letters]
    return '\n'.join(mapped_descriptions)

<b>main</b> This function runs each risk checking function and outputs one dataframe.<br>
The `result` output is a boolean matrix representing the status of each risk factor <br>
for a particular transaction. <br>
<br>
The function can be configured to output additional information regarding the specific risk ratings<br>
for transactions.<br>
<br>

In [21]:
def main():
    result = pd.DataFrame()

    print('Running risk functions...')
    # running risk checkers against columns
    result['a'] = a_checker(data['Accounting Codes'])
    result['b'] = b_checker(data[['PO Date', 'BOS Date', 'Invoice Date']])
    result['c'] = c_checker(bos_exist, data[['BOS Quantity', 'BOS Cost Per Unit', 'Invoice Quantity','Invoice Cost Per Unit']])
    result['d'] = d_checker(bos_exist, data[['BOS Quantity', 'BOS Cost Per Unit', 'PO Quantity', 'PO Cost Per Unit']])
    result['e'] = e_checker(data['PO Quantity'], data['Invoice Quantity'])
    result['g'] = g_checker(po_exist, bos_exist, inv_exist, data['Accounting Codes'])
    result['h'] = h_checker(data)
    result['i'] = i_checker(data)
    result['j'] = j_checker(data, 'Invoice Amount Paid', 10000, 100000)
    result['k'] = k_checker(data, 'Invoice Amount Paid', 999, 4999, 14999, 34999, 49999, 99999)
    result['l'] = l_checker(data["Check Number"], data['PO Quantity'], data['BOS Quantity'], data['Invoice Quantity'])
    result['m'] = m_checker(data)
    result['n'] = n_checker(data)
    # result['o'] = o_checker(data)

    print('Calculating results...')
    risk_rating = pd.DataFrame()
    risk_rating['a'] = result['a'] * 18
    risk_rating['b'] = result['b'] * 62
    risk_rating['c'] = result['c'] * 67
    risk_rating['d'] = result['d'] * 73
    risk_rating['e'] = result['e'] * 64
    risk_rating['g'] = result['g'] * 77
    risk_rating['h (department score)'] = result['h'] * 1
    risk_rating['i'] = result['i'] * 54
    risk_rating['j'] = result['j'] * 64
    risk_rating['k'] = result['k'] * 42
    risk_rating['l'] = result['l'] * 84
    risk_rating['m'] = result['m'] * 63
    risk_rating['n'] = result['n'] * 58
    # risk_rating['o'] = result['o'] * 37

    letters = pd.DataFrame()
    letters['a'] = result['a'].replace({True: 'a', False: ''})
    letters['b'] = result['b'].replace({True: 'b', False: ''})
    letters['c'] = result['c'].replace({True: 'c', False: ''})
    letters['d'] = result['d'].replace({True: 'd', False: ''})
    letters['e'] = result['e'].replace({True: 'e', False: ''})
    letters['g'] = result['g'].replace({True: 'g', False: ''})
    letters['i'] = result['i'].replace({True: 'i', False: ''})
    letters['j'] = result['j'].replace({True: 'j', False: ''})
    letters['k'] = result['k'].replace({True: 'k', False: ''})
    letters['l'] = result['l'].replace({True: 'l', False: ''})
    letters['m'] = result['m'].replace({True: 'm', False: ''})
    letters['n'] = result['n'].replace({True: 'n', False: ''})

    # out of 796 total (a-o)
    letters['risks'] = letters.astype(str).agg(''.join, axis=1)
    letters['Description'] = letters['risks'].apply(map_descriptions)
    sums = risk_rating.sum(axis=1).rename('Risk Rating')


    highlighted = pd.concat([risk_rating['h (department score)'], letters['risks'], sums, letters['Description']], axis=1)
    slice_ = ['cluster', 'h (department score)', 'risks', 'Risk Rating', 'Description']
    concatenated_data = pd.concat([data.drop('Processed Invoice Description', axis=1), highlighted], axis=1)
    concatenated_data = concatenated_data.style.set_properties(**{'background-color': 'yellow'}, subset=slice_)
        
    ####################################################
    ### uncomment lines for the information you want ###
    ####################################################
    
    """data will show the underlying data we have been using"""
    # print(data.head())
    
    """sums will show the transactions with their appropriate risk rating scores"""
    # print(sums.head())
    
    """result will show the transactions with their appropriate risk categories"""
    # print(result.head())

    #export final data
    print('Exporting data...')
    concatenated_data.to_excel('Risk_Result.xlsx', index=False, engine='openpyxl')
    print('Done!')

In [24]:
if __name__ == '__main__':
    main()

Running risk functions...
Calculating results...
Exporting data...
Done!
